In [14]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

In [15]:
!pip install h5py

In [16]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [17]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'CE' else 0)

In [18]:
df = df.set_index('image_id')

In [19]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [20]:
final_data= pd.read_csv('../input/final-preprocessing/navigation_v2.csv')
final_data = final_data.set_index('Unnamed: 0')
final_data = final_data.astype(object)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124,2125,2126,2127,2128,2129,2130,2131,2132,2133,2134,2135,2136,2137,2138,2139,2140,2141,2142,2143,2144,2145,2146,2147,2148,2149,2150,2151,2152,2153,2154,2155,2156,2157,2158,2159,2160,2161,2162,2163,2164,2165,2166,2167,2168,2169,2170,2171,2172,2173,2174,2175,2176,2177,2178,2179,2180,2181,2182,2183,2184,2185,2186,2187,2188,2189,2190,2191,2192,2193,2194,2195,2196,2197,2198,2199,2200,2201,2202

In [21]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (512, 512))

In [22]:
# def get_patch2(os_obj, tl_pixel, patch_shape):
#     return np.array(os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB"))

# final_data = pd.DataFrame()
# ids = []
# for n in range(df.shape[0]):
#     img_path = "../input/mayo-clinic-strip-ai/train/"+df.iloc[n]['image_id']+".tif"
#     label = df.iloc[n]['label']
#     i_id  = df.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     print(n)
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             im = get_patch2(img , (i,j), (512,512) )
#             if im.std()>15.0:
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_data = pd.concat([final_data , series],axis=1)
#     final_data.columns = ids


In [23]:
global train_test_list
train_test_list = np.random.choice(np.arange(754),754)

In [24]:
def data_gen(df = df , data = final_data):
    for i in train_test_list[:650]:
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            for x in range(3):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            for x in range(1):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
       
    
    
def test_gen(df = df , data = final_data):
    for i in train_test_list[650:]:
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)

In [25]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)
testset = tf.data.Dataset.from_generator(
     test_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)

In [26]:

dataset = dataset.batch(4)
dataset=dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size = 10, seed=3)
testset = testset.batch(4)
testset=testset.prefetch(1)

In [27]:
efficient = tf.keras.applications.EfficientNetB1(
    include_top=False,
    weights='imagenet',
    input_shape=(512,512,3),
)
def augmentation(A):
    A = tf.keras.layers.RandomFlip()(A)
    A =tf.keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    A = tf.keras.layers.RandomZoom(
        (-0.1,0.1),
        (-0.1,0.1),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    return A

In [28]:
inputA = keras.layers.Input(shape=(512,512,3),name='input_71')
inputB = keras.layers.Input(shape=(512,512,3),name='input_72')
inputC = keras.layers.Input(shape=(512,512,3),name='input_73')
inputD = keras.layers.Input(shape=(512,512,3),name='input_74')
inputE = keras.layers.Input(shape=(512,512,3),name='input_75')

A = augmentation(inputA)
A = efficient(A)
A = keras.models.Model(inputs=inputA, outputs=A)
B = augmentation(inputB)
B = efficient(B)
B = keras.models.Model(inputs=inputB, outputs=B)
C = augmentation(inputC)
C = efficient(C)
C = keras.models.Model(inputs=inputC, outputs=C)
D = efficient(inputD)
D = keras.models.Model(inputs=inputD, outputs=D)
E = efficient(inputE)
E = keras.models.Model(inputs=inputE, outputs=E)

X = keras.layers.Concatenate(axis=-1)([A.output, B.output, C.output, D.output, E.output])
X = keras.layers.Conv2D(512, 4, strides = 4)(X) 
X = keras.layers.Conv2D(256, 2, strides = 2)(X)
X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)
X = keras.layers.Dense(1024 ,activation = 'relu')(X)
X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(128 ,activation = 'relu')(X)
X = keras.layers.Dense(64 ,activation = 'relu')(X)
X = keras.layers.Dense(32 ,activation = 'relu')(X)
X = keras.layers.Dense(16 ,activation = 'relu')(X)
X = keras.layers.Dense(8 ,activation = 'relu')(X)
X = keras.layers.Dense(1 ,activation = 'sigmoid')(X)
                                               
model = keras.models.Model(inputs=[A.input,B.input,C.input,D.input,E.input], outputs=X)

In [29]:
model.summary()

Model: "model_35"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_72 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
random_flip_15 (RandomFlip)     (None, 512, 512, 3)  0           input_71[0][0]                   
___________________________________________________________________________________________

In [38]:
model.load_weights("./model3/model").expect_partial()

In [39]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.00001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    label_smoothing=0.05,
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)

In [40]:
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])

In [32]:
!mkdir model3

In [41]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model3/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [42]:
H = model.fit(dataset, validation_data=testset, epochs=3, callbacks=[model_checkpoint_callback_LASSO])

Epoch 1/3


2022-09-26 18:38:53.072410: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 5 of 10
2022-09-26 18:39:03.585979: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


251/251 [==============================] - 766s 3s/step - loss: 0.5957 - recall_2: 0.5711 - precision_2: 0.7402 - auc_2: 0.7387 - val_loss: 0.6171 - val_recall_2: 1.0000 - val_precision_2: 0.7282 - val_auc_2: 0.5914
Epoch 2/3


2022-09-26 18:49:55.634623: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 5 of 10
2022-09-26 18:50:05.918236: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


251/251 [==============================] - 654s 3s/step - loss: 0.5719 - recall_2: 0.6250 - precision_2: 0.7945 - auc_2: 0.7870 - val_loss: 0.6325 - val_recall_2: 0.9867 - val_precision_2: 0.7255 - val_auc_2: 0.6069
Epoch 3/3


2022-09-26 19:00:48.752537: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 5 of 10
2022-09-26 19:00:58.848672: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.


251/251 [==============================] - 633s 2s/step - loss: 0.5759 - recall_2: 0.6272 - precision_2: 0.7802 - auc_2: 0.7719 - val_loss: 0.6207 - val_recall_2: 1.0000 - val_precision_2: 0.7282 - val_auc_2: 0.5986


In [ ]:
!mkdir final_model

In [ ]:
!mkdir best_model

In [ ]:
model.save('./final_model/model')

In [ ]:
model.load_weights('./model3/model').expect_partial()
model.save('./best_model/model')

In [ ]:
model.evaluate(testset)

In [ ]:
# def get_patch1(img, tl_pixel, patch_shape):
#     img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
#     return img

# final_samples= pd.DataFrame()
# ids = []
# for n in tqdm(range(samples.shape[0])):
#     img_path = "../input/mayo-clinic-strip-ai/test/"+samples.iloc[n]['image_id']+".tif"
#    #label = samples.iloc[n]['label']
#     i_id = samples.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     n = 0
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             if n ==20:
#                 break           
#             im = get_patch1(img , (i,j), (512,512) )
#             if im.std()>10.0:
#                 n=n+1
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_samples = pd.concat([final_samples , series],axis=1)
#     final_samples.columns = ids
# final_samples = final_samples.T


In [ ]:
# final_samples

In [ ]:
# samples

In [ ]:
# def sample_gen(df = samples , data = final_samples):
#     for i in range(samples.shape[0]):
#         img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][0]),(512,512))
#         img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][1]),(512,512))
#         img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][2]),(512,512))
#         img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][3]),(512,512))
#         img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][4]),(512,512))
#         yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5})
# sampleset = tf.data.Dataset.from_generator(
#      sample_gen,
#      ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}),
#     ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}))


In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")

In [ ]:
# weight_file = "./model"
# model.load_weights(weight_file).expect_partial()
# print("Weights loaded successfully")